In [6]:
# Nice notebook environment
%matplotlib inline
from IPython.display import display

# Importing numpy
import numpy as np
from numpy import *
import torch
import torch.nn as nn
import dgl
from dgl.nn import GraphConv, SumPooling, AvgPooling
from dgl.nn.pytorch import Sequential

# Importing matplotlib for graphics and fixing the default size of plots
import matplotlib
from matplotlib import rc
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors # para colores en las gráficas
from matplotlib.ticker import AutoMinorLocator

matplotlib.rcParams['mathtext.fontset']='cm'
matplotlib.rcParams.update({'font.size': 18})
plt.rcParams["figure.figsize"] = (10,8)

# In case we need sympy
from sympy import init_printing
init_printing(use_latex=True)

# For working with well stabished parameters
from mendeleev import element
from scipy.constants import physical_constants

import tarfile   # para tratar los archivos .tar
import glob      # para buscar directorios
import re        # para expresiones regulares

In [7]:
class LinearDens(nn.Module):
    
    r"""Applies a linear transformation to the incoming data: :math:`y = xA^T + b`
    
    LinearDens is introduced as a way of use nn.Linear together with GraphConv since Sequential() 
    requires the layers to be joined have the same input. In this case, a dgl.graph and a features tensor.
    """
    def __init__(self, in_feats, out_feats):
    
        super(LinearDens, self).__init__()
    
        self.linear = nn.Linear(in_feats, out_feats)
    
    def forward(self, graph, feat, weight=None, edge_weight=None):
        
        return self.linear(feat)

In [8]:
def graph_potential(n_gc_layers: int = 1,
                    n_fc_layers: int = 1,
                    activation_function: str = 'Tanh',
                    pooling: str = 'add',
                    n_node_features: int = 55,
                    n_edge_features: int = 1,
                    n_neurons: int = None ) -> torch.nn.Module: 

    r""" This is a function that returns a model with the desired number of Graph-Convolutional (n_gc_layers) layers and 
    Fully-Connected (n_fc_layers) linear layers, using the specified non-linear activation layers interspaced
    between them. 

    Args:
        
        n_gc_layers (int): number of Graph-Convolutional layers (GraphConv from dgl). Default: 1 
        n_fc_layers (int): number of densely-connected -Fully Connected- linear layers (LinearDens). Default: 1
        activation_function (char): nonlinear activation layer; can be any activation available in torch. Default: Tanh
        pooling (char): the type of pooling of node results. If set to 'add' performs a sum over all nodes (e.g. return total energy).
                        In other case, performs the 'mean' to return energy per atom. Default: Add
        n_node_features (int): length of node feature vectors. Default: 55
        
        
        n_edge_features (int): length of edge feature vectors; currently 1, the distance.
        ATENCION PQ CON LA FUNCION QUE ESTAMOS USANDO NO ES NECESARIO SABER LOS EDGES
        
        
        n_neurons (int): number of neurons in deep layers (if they exist) in the densely-connected network.  
                       If set to None, it is internally set to n_node_features. Default: None

    """


    # First, we define the activation layer, the pooling layer and the number of neurons:
    
    activation_layer = eval("torch.nn." + activation_function + "()")
    
    if pooling == 'add':
        poolingLayer = SumPooling()
   
    else:
        poolingLayer = AvgPooling()

    if n_neurons == None:
        n_neurons = n_node_features

    
    # Now, the Graph-convolutional layers: 

    conv = GraphConv(in_feats = n_node_features, out_feats = n_node_features, activation = activation_layer)
    
    model = conv
      
    for n in range(1, n_gc_layers-1):
        
        model = Sequential(model, conv)
    
    
    
    # Then the fully-connected layers: 

    linear1 = LinearDens(n_node_features, 1)           # only 1 fully connected layer; last conv layer --> final result)
    linear2 = LinearDens(n_node_features, n_neurons)   # conv layer --> fully connected layer
    linear3 = LinearDens(n_neurons, n_neurons)         # fully connected layer --> fully connected layer
    linear4 = LinearDens(n_neurons, 1)                 # fully connected layer --> final result
    
    
    if n_fc_layers == 1:
        
        model = Sequential(model, linear1)
    
    else:
        
        model = Sequential(model, linear2)
        
        model = Sequential(model, activation_layer)
        
        for n in range(1, n_fc_layers-1):
            
            model = Sequential(model, linear3)
            
            model= Sequential(model, activation_layer)
            
        # and finally the exit layer

        model = Sequential(model, linear4)
    
    # last but not least, the pooling layer

    model = Sequential(model, poolingLayer)
    
    return model

In [12]:
####PARTE DE PRUEBAS#####


#Creamos un grafo sencillo
g = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]), num_nodes=6)
g = dgl.add_self_loop(g)
#print(g.num_nodes())

g.ndata['fit'] = torch.ones(g.num_nodes(), 3)
print('n_node_features =', g.ndata['fit'].shape[1])
g.edata['a'] = torch.randn(g.num_edges(), 4)
print('Nodes g:\n',g.ndata['fit'])

activation_function = 'Tanh'
activation_layer = eval("torch.nn." + activation_function + "()")

conv = GraphConv(in_feats = 3, out_feats = 3, activation= activation_layer)  # allow_zero_in_degree = True
conv2 = GraphConv(in_feats = 3, out_feats = 2, activation= activation_layer)
lin = LinearDens(2, 1)

res = conv(g, g.ndata['fit'])
#print(res)
res = conv(g, res)
#print(res)
res = conv2(g, res)
#print(res)
res = lin(g, res)
#print(res)



model = Sequential(conv, conv)
model = Sequential(model, conv2)
model = Sequential(model, lin)
res = model(g, g.ndata['fit'])
#print(res)


model2 = graph_potential(n_gc_layers = 3, n_fc_layers = 1, n_node_features = g.ndata['fit'].shape[1], n_edge_features=3)
res = model2(g, g.ndata['fit'])
print(res)

n_node_features = 3
Nodes g:
 tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[0.0841, 0.1991, 0.3221],
        [0.2027, 0.4560, 0.6721],
        [0.2027, 0.4560, 0.6721],
        [0.2027, 0.4560, 0.6721],
        [0.2027, 0.4560, 0.6721],
        [0.2027, 0.4560, 0.6721]], grad_fn=<TanhBackward0>)
tensor([[-0.0154,  0.0007,  0.0947],
        [-0.0639,  0.0248,  0.4009],
        [-0.0639,  0.0248,  0.4009],
        [-0.0639,  0.0248,  0.4009],
        [-0.0639,  0.0248,  0.4009],
        [-0.0639,  0.0248,  0.4009]], grad_fn=<TanhBackward0>)
tensor([[0.0779],
        [0.1495],
        [0.1495],
        [0.1495],
        [0.1495],
        [0.1495]], grad_fn=<AddmmBackward0>)
tensor([[0.0779],
        [0.1495],
        [0.1495],
        [0.1495],
        [0.1495],
        [0.1495]], grad_fn=<AddmmBackward0>)
tensor([[-0.2287]], grad_fn=<SegmentReduceBackward>)
